STEP 1 = Importig data and required modules .

In [ ]:
import numpy as np
import pandas as pd
import io
import tensorflow as tf
from tensorflow.keras.preprocessing import text , sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D ,MaxPooling1D
from sklearn.model_selection import train_test_split
print(tf.__version__)

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

2.2.0
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving datasets_483_982_spam.csv to datasets_483_982_spam (2).csv


In [ ]:
data = pd.read_csv(io.BytesIO(uploaded['datasets_483_982_spam.csv']), encoding='latin-1')

In [ ]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


Step 2 = Preprocessing Data
* Removing null values
* Cleaning Columns
* Removing punctation and stopwords
* Doing tokenization & vectorization
* Calculating importance of words by tfdif

In [ ]:
data.isnull().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [ ]:
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":"label", "v2":"text"})

Label encoding
- Label of texts should be in integer ,it is important to apply model.fit function.
- If labels are not given , use the below code:-
                  from sklearn.preprocessing import LabelEncoder   
                  Encoder = LabelEncoder()
                  Train_Y = Encoder.fit_transform(Train_Y)
                  Test_Y = Encoder.fit_transform(Test_Y)

In [ ]:
data['spam'] = data['label'].map( {'spam': 1, 'ham': 0} ).astype(int)
data.head(15)


,label,text,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1
6,ham,Even my brother is not like to speak with me. ...,0
7,ham,As per your request 'Melle Melle (Oru Minnamin...,0
8,spam,WINNER!! As a valued network customer you have...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1


In [ ]:

data_ham  = data[data['spam'] == 0].copy()
data_spam = data[data['spam'] == 1].copy()

Preprocessing

In [ ]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
#Stop words are words like “and”, “the”, “him”, which are presumed to be uninformative & non-predicative , so avoided
stopwords.words("english")[100:110]                   # to see some of the stopwords

['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']

Function to remove punctations and stopwords

In [ ]:
def remove_punctuation_and_stopwords(sms):
    
    sms_no_punctuation = [ch for ch in sms if ch not in string.punctuation]
    sms_no_punctuation = "".join(sms_no_punctuation).split()
    
    sms_no_punctuation_no_stopwords = \
        [word.lower() for word in sms_no_punctuation if word.lower() not in stopwords.words("english")]
        
    return sms_no_punctuation_no_stopwords

In [ ]:
data['text'].apply(remove_punctuation_and_stopwords).head()


0    [go, jurong, point, crazy, available, bugis, n...
1                       [ok, lar, joking, wif, u, oni]
2    [free, entry, 2, wkly, comp, win, fa, cup, fin...
3        [u, dun, say, early, hor, u, c, already, say]
4    [nah, dont, think, goes, usf, lives, around, t...
Name: text, dtype: object

In [ ]:

# Countvectorizer = https://www.educative.io/edpresso/countvectorizer-in-python

CountVectorizer :-   
In order to use textual data for predictive modeling, the text must be given some no or token – this process is called tokenization. These words need to then be encoded as integers/floating-point values, for use as inputs in machine learning algorithms. This process is called feature extraction (or vectorization).

Scikit-learn’s CountVectorizer is used to convert a collection of text documents to a vector of term/token counts.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer = CountVectorizer(analyzer = remove_punctuation_and_stopwords).fit(data['text'])

In [ ]:
print(len(bow_transformer.vocabulary_))
# It will print total no of words we have in our vocalbury

9431


In [ ]:
# Applying vocalbury to all words , here vocalbury = bow-transformer
bow_data = bow_transformer.transform(data['text'])

TFIDF = A short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(bow_data)

In [ ]:
# To check the importanct of words from a ham label sms .
sample_ham = data['text'][4]
bow_sample_ham = bow_transformer.transform([sample_ham])
tfidf_sample_ham = tfidf_transformer.transform(bow_sample_ham)
print(tfidf_sample_ham)

  (0, 8664)	0.4000611457714059
  (0, 8243)	0.35462891367407595
  (0, 8224)	0.2665748147939407
  (0, 5631)	0.40493594386802356
  (0, 5004)	0.44910913790937335
  (0, 3762)	0.35674330934716636
  (0, 2882)	0.23016324066220836
  (0, 1279)	0.31179198007951625


Step = 3

Train test split for tfdif matrix only (here) :-

In [ ]:
data_tfidf = tfidf_transformer.transform(bow_data)

data_tfidf_train, data_tfidf_test, label_train, label_test = \
    train_test_split(data_tfidf, data["spam"], test_size=0.3, random_state=5)

In [ ]:
data_tfidf_test

<1672x9431 sparse matrix of type '<class 'numpy.float64'>'
	with 14647 stored elements in Compressed Sparse Row format>

In [ ]:
data['length'] = data['text'].apply(len)

from scipy.sparse import  hstack
X2 = hstack((data_tfidf ,np.array(data['length'])[:,None])).A

In [ ]:
X2_train, X2_test, y2_train, y2_test = \
    train_test_split(X2, data["spam"], test_size=0.3, random_state=5)

In [ ]:
from sklearn.model_selection import train_test_split

data_train, data_test, label_train, label_test = \
    train_test_split(data_tfidf, data["spam"], test_size=0.3, random_state=5)

Step 4 = Add functions to measure accuracy

In [ ]:
# To print classification report and accuracy 
def print_validation_report(y_true, y_pred):
    print("Classification Report")
    print(classification_report(y_true, y_pred))
    acc_sc = accuracy_score(y_true, y_pred)
    print("Accuracy : "+ str(acc_sc))
    
    return acc_sc
# To plot confusion matrix
def plot_confusion_matrix(y_true, y_pred):
    mtx = confusion_matrix(y_true, y_pred)
    #fig, ax = plt.subplots(figsize=(4,4))
    sns.heatmap(mtx, annot=True, fmt='d', linewidths=.5,  
                cmap="Blues", cbar=False, ax=ax)
    #  square=True,
    plt.ylabel('true label')
    plt.xlabel('predicted label')

#Step = 5

##Classifiers


###1. Naive bayes classifier

Naive Bayes is a family of statistical algorithms we can make use of when doing text classification. One of the members of that family is Multinomial Naive Bayes (MNB). One of its main advantages is that you can get really good results when data available is not much (~ a couple of thousand tagged samples) and computational resources are scarce.

All you need to know is that Naive Bayes is based on Bayes’s Theorem, which helps us compute the conditional probabilities of occurrence of two events based on the probabilities of occurrence of each individual event. This means that any vector that represents a text will have to contain information about the probabilities of appearance of the words of the text within the texts of a given category so that the algorithm can compute the likelihood of that text’s belonging to the category.

In [ ]:
data_train = data_train.A
data_test = data_test.A

In [ ]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(data_train, label_train)
pred_test_MNB = spam_detect_model.predict(data_test)

acc_MNB = accuracy_score(label_test, pred_test_MNB)
print(acc_MNB)

0.965311004784689


###2. SVM Classifier

Support Vector Machines (SVM) is just one out of many algorithms we can choose from when doing text classification. Like naive bayes, SVM doesn’t need much training data to start providing accurate results. Although it needs more computational resources than Naive Bayes, SVM can achieve more accurate results.

In [ ]:
from sklearn import model_selection, naive_bayes, svm
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(data_train, label_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
pred_SVM = SVM.predict(data_test)
acc_SVM = accuracy_score(label_test,pred_SVM)
print(acc_SVM)


0.9838516746411483


###3. Deep Learning
Deep learning is a set of algorithms and techniques inspired by how the human brain works. The two main deep learning architectures used in text classification are Convolutional Neural Networks (CNN) and Recurrent Neural Networks (RNN).

- On the one hand, deep learning algorithms require much more training data than traditional machine learning algorithms, i.e. at least millions of tagged examples. On the other hand, traditional machine learning algorithms such as SVM and NB reach a certain threshold where adding more training data doesn’t improve their accuracy. In contrast, deep learning classifiers continue to get better the more data you feed them with.

- Deep learning algorithms such as Word2Vec or GloVe are also used in order to obtain better vector representations for words and improve the accuracy of classifiers trained with traditional machine learning algorithms(transfered learning)

In [ ]:
train_df = data.copy()
train_df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
x= train_df['v2'].values
x

array(['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       'Ok lar... Joking wif u oni...',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       ..., 'Pity, * was in mood for that. So...any other suggestions?',
       "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
       'Rofl. Its true to its name'], dtype=object)

In [ ]:
def remove_stopwords(x):
    
     x_no_stopwords = \
     [word.lower() for word in x if word.lower() not in stopwords.words("english")]
        
     return x_no_stopwords

In [ ]:
x = remove_stopwords(x)


In [ ]:
y = train_df['v1'].map( {'spam': 1, 'ham': 0} ).astype(int)
y.head()

0    0
1    0
2    1
3    0
4    0
Name: v1, dtype: int64

Data preparation

In [ ]:
max_features = 20000 # most frequent 20000 words appearing
max_text_length= 200
x_tokenizer = text.Tokenizer(max_features)
x_tokenizer.fit_on_texts(list(x))
x_tokenized = x_tokenizer.texts_to_sequences(x)
x_train_val = sequence.pad_sequences(x_tokenized, maxlen=max_text_length)


Preparing Embedding matrix

In [ ]:
!wget nlp/stanford.edu/data/glove.6b.zip
!unzip -q glove.6b.zip

--2020-07-26 16:55:37--  http://nlp/stanford.edu/data/glove.6b.zip
Resolving nlp (nlp)... failed: Name or service not known.
wget: unable to resolve host address ‘nlp’
unzip:  cannot find or open glove.6b.zip, glove.6b.zip.zip or glove.6b.zip.ZIP.


In [ ]:
embedding_dims = 100
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs
f.close()

embedding_matrix = np.zeros((max_features, embedding_dims))
for word, index in x_tokenizer.word_index.items():
  if index > max_features -1:
    break
  else:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[index] = embedding_vector

In [ ]:
print('Build model...')
model = Sequential()
# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
#load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
#(we don't want to update them during training).
model.add(Embedding(max_features,
                    embedding_dims,
                    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                    trainable=False))
model.add(Dropout(0.2))

Build model...


In [ ]:
filters = 250
kernel_size = 3
hidden_dims = 250

In [ ]:
# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(filters,
                 5,
                 padding='valid',
                 activation='relu'))
# we use max pooling:
model.add(GlobalMaxPooling1D())
# We add a vanilla hidden layer:
model.add(Dense(hidden_dims, activation='relu'))
model.add(Dropout(0.2))

# We project onto 6 output layers, and squash it with a sigmoid:
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         2000000   
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 250)         75250     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 250)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 250)         312750    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 250)              

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y, test_size=0.15, random_state=1)

In [ ]:
batch_size = 32
epochs = 2
validate_on =(x_val,y_val)

In [ ]:
validate_on

(array([[   0,    0,    0, ..., 1628,    2,  130],
        [   0,    0,    0, ...,    2,    6,  209],
        [   0,    0,    0, ...,    5,  153,  316],
        ...,
        [   0,    0,    0, ...,    2,  359,  450],
        [   0,    0,    0, ...,   43,   41,  203],
        [   0,    0,    0, ...,  423,   20,    9]], dtype=int32), 1078    0
 4028    0
 958     0
 4642    0
 4674    1
        ..
 2902    0
 1387    0
 4300    0
 372     0
 4394    0
 Name: v1, Length: 836, dtype: int64)

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=validate_on)

Epoch 1/2
148/148 [==============================] - 25s 172ms/step - loss: 0.1956 - accuracy: 0.9267 - val_loss: 0.1090 - val_accuracy: 0.9641
Epoch 2/2
148/148 [==============================] - 25s 170ms/step - loss: 0.0992 - accuracy: 0.9662 - val_loss: 0.1196 - val_accuracy: 0.9605


27/27 [==============================] - 1s 48ms/step


ValueError: ignored